# Setup
- packages installation  
- imports

In [26]:
%pip install numpy pandas matplotlib opencv-python

In [27]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [28]:
import cv2
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import os
import random

# UTIL

In [29]:
DATASET_CARS_BR_PATH = "./dataset_RodoSol-ALPR/images/cars-br" # Path to images of cars with old license plate model
DATASET_CARS_ME_PATH = "./dataset_RodoSol-ALPR/images/cars-me" # Path to images of cars with MERCOSUL license plate model
DATASET_MOTORCYCLES_BR_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-br" # Path to images of motorcycles with old license plate model
DATASET_MOTORCYCLES_ME_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-me" # Path to images of motorcycles with MERCOSUL license plate model

VERBOSE = True
DEV_MODE = False

In [30]:
def verbose_decorator(func):
    def wrapper(*args, **kwargs):
        if VERBOSE:
            print()
            print(func)
        return func(*args, **kwargs)
    return wrapper

In [31]:
@verbose_decorator
def unit_exec(image_path, save_path, algorithm, algorithm_args):
    """
    unit_exec does a test for dev purposes. It receives two paths. First path is where the image to be processed is.
    The second one is where to save it. This function is for comparing algorithms and its parameters.
    """
    image = cv2.imread(image_path)
    image = algorithm(image, **algorithm_args)
    cv2.imwrite(save_path, image)

In [32]:
@verbose_decorator
def sample_images(data_path: str, exclude_path: str, sample_size=None) -> list[tuple]:
    """
    sample_images randomly chooses a number of images, equal to the parameter sample_size, from a directory specified by data_path 
    excluding images already at directory specified by exclude_path. If sample_size is None then pick all images from directory.
    It returns a list containing tuples, each tuple with a opencv image and its OS path.
    """
    
    if exclude_path == "" or exclude_path is None: # If not informed then ignore excluding.
        excluded_images = []
    else:
        excluded_images= [data_path + '/' + file_name for file_name in os.listdir(exclude_path) if file_name.endswith(".jpg")]

    image_paths = [data_path + '/' + file_name for file_name in os.listdir(data_path) if file_name.endswith(".jpg") and file_name not in excluded_images]
    if sample_size is not None:
        image_paths = random.sample(image_paths, min(sample_size, len(image_paths)))
    
    images = [(cv2.imread(image_path), image_path) for image_path in image_paths]
    if VERBOSE:
        print(f"Excluded files from sampling: {excluded_images}")
        print(f"Sampled files: {image_paths}")
        
    
    return images

In [33]:
@verbose_decorator
def combine_images_side_by_side(image1, image2):
    # Determine the maximum height of the two images
    max_height = max(image1.shape[0], image2.shape[0])
    
    # Create a canvas with white background
    combined_height = max_height
    combined_width = image1.shape[1] + image2.shape[1]
    combined_image = np.ones((combined_height, combined_width, 3), dtype=np.uint8) * 255  # Fill with white (255, 255, 255)
    
    # Paste the first image onto the canvas
    combined_image[:image1.shape[0], :image1.shape[1]] = image1
    
    # Paste the second image onto the canvas, starting from the width of the first image
    combined_image[:image2.shape[0], image1.shape[1]:] = image2
    
    return combined_image

@verbose_decorator
def compare_and_save(original_images, processed_images, save_path):
   os.makedirs(save_path, exist_ok=True) # Create directory and dont raise error if already exists
   
   images = [(orig, proc) for orig, proc in zip(original_images, processed_images)]

   for orig, proc in images:
       combined_images = combine_images_side_by_side(orig[0], proc[0])
       image_save_path = save_path + '/' + orig[1].split('/')[-1]
       cv2.imwrite(image_save_path, combined_images)
       
       if VERBOSE:
            print(f"{orig[1].split('/')[-1]} saved at {image_save_path}")
           


In [34]:
@verbose_decorator
def show_comparison(original_images, processed_images):
    """
    show_comparison shows a comparison side by side between original_images and processed_images
    """
    images = [(orig, proc) for orig, proc in zip(original_images, processed_images)]
    if VERBOSE:
        print(f"images zipped structure: {[(orig[1], proc[1]) for orig, proc in images]}")
        for orig, proc in images:
            print(f"{orig[1].split('/')[-1]} shape: {orig[0].shape}")
            print(f"{proc[1]} shape: {proc[0].shape}")
    
    fig, axes = plt.subplots(len(images), 2, figsize=(20, 10))
    ax = axes.ravel()
    
    ax_i = -1
    for orig, proc in images:
        ax_i += 1
        ax[ax_i].imshow(orig[0])
        ax[ax_i].set_title(orig[1].split('/')[-1] + " Original")
        ax[ax_i].set_axis_off()
        
        ax_i += 1
        ax[ax_i].imshow(proc[0])
        ax[ax_i].set_title(proc[1].split('/')[-1] + " Processed")
        ax[ax_i].set_axis_off()


In [35]:
@verbose_decorator
def sample_execute_save_show(data_path: str, save_path: str, algorithm: callable, algorithm_args: dict, show=False, pre_sample=None, sample_size=None):
    """
    sample_execute_save_show randomly sample images from directory specified by data_path ignoring 
    images already at directory specified by save_path, execute a algorithm of image processing which must return a single image,
    save processed images at a directory specified by save_path and then shows a comparison between the original and processed image.
    If presample contains a valid images names list then execute algorithm with this images.
    If sample_size is None then execute algorithm on the entire directory.
    """
    os.makedirs(save_path, exist_ok=True) # Create directory and dont raise error if already exists
    is_pre_sample = isinstance(pre_sample, list) and all([sample in os.listdir(data_path) for sample in pre_sample]) # If is a list and directory specified by data_path contain ALL samples.
    
    if VERBOSE:
        print(f"is pre sampled: {is_pre_sample}")
    
    # Defining algorithm input scope
    if is_pre_sample: # Images pre sampled
        original_images = pre_sample
    elif sample_size is None:
        original_images = sample_images(data_path, save_path, sample_size=None)
    else: # Sample Images
        original_images = sample_images(data_path, save_path, sample_size)

    processed_images = []
    for original_image, path in original_images:
        image_save_path = save_path + '/' + path.split('/')[-1]

        processed_image = algorithm(original_image, **algorithm_args)
        processed_images.append( (processed_image, image_save_path) )

        cv2.imwrite(image_save_path, processed_image)
        if VERBOSE:
            print(f"{path.split('/')[-1]} saved at {image_save_path}")

    if show:
        show_comparison(original_images, processed_images)

    return processed_images

# Applying Grayscale, equalizing histogram and normalizing brightness


In [36]:
SAVE_CARS_BR_GRAYSCALE_PATH = "./output/images/grayscale/cars-br" # Save path to images of cars with old license plate model converted to grayscale
SAVE_CARS_ME_GRAYSCALE_PATH = "./output/images/grayscale/cars-me" # Save path to images of cars with MERCOSUL license plate model converted to grayscale
SAVE_MOTORCYCLES_BR_GRAYSCALE_PATH = "./output/images/grayscale/motorcycles-br" # Save path to images of motorcycles with old license plate model converted to grayscale
SAVE_MOTORCYCLES_ME_GRAYSCALE_PATH = "./output/images/grayscale/motorcycles-me" # Save path to images of motorcycles with MERCOSUL license plate model converted to grayscale


SAVE_CARS_BR_HISTEQ_PATH = "./output/images/histeq/cars-br" # Save path to images of cars with old license plate model with histogram equalized
SAVE_CARS_ME_HISTEQ_PATH = "./output/images/histeq/cars-me" # Save path to images of cars with MERCOSUL license plate model with histogram equalized
SAVE_MOTORCYCLES_BR_HISTEQ_PATH = "./output/images/histeq/motorcycles-br" # Save path to images of motorcycles with old license plate model with histogram equalized
SAVE_MOTORCYCLES_ME_HISTEQ_PATH = "./output/images/histeq/motorcycles-me" # Save path to images of motorcycles with MERCOSUL license plate model with histogram equalized


In [37]:
def convert_to_grayscale(image):
    """
    Convert an OpenCV image to grayscale.

    Args:
        image (numpy.ndarray): The input OpenCV image.

    Returns:
        numpy.ndarray: The grayscale image.
    """
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

In [38]:
def equalize_histogram(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(grayscale_image)
    
    return equalized_image

In [39]:
if not DEV_MODE and not os.path.exists("./output/images/grayscale/"): # If grayscale was not executed yet
    # GRAYSCALE
    _ = sample_execute_save_show(data_path=DATASET_CARS_BR_PATH, 
                                save_path=SAVE_CARS_BR_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_CARS_ME_PATH, 
                                save_path=SAVE_CARS_ME_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_MOTORCYCLES_BR_PATH, 
                                save_path=SAVE_MOTORCYCLES_BR_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_MOTORCYCLES_ME_PATH, 
                                save_path=SAVE_MOTORCYCLES_ME_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

In [40]:
if not DEV_MODE and not os.path.exists("./output/images/histeq/"): # If histogram equalization was not executed yet
    # EQUALIZING HISTOGRAM
    _ = sample_execute_save_show(data_path=SAVE_CARS_BR_GRAYSCALE_PATH, 
                                save_path=SAVE_CARS_BR_HISTEQ_PATH,
                                algorithm=equalize_histogram,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=SAVE_CARS_ME_GRAYSCALE_PATH, 
                                save_path=SAVE_CARS_ME_HISTEQ_PATH,
                                algorithm=equalize_histogram,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_BR_GRAYSCALE_PATH, 
                                save_path=SAVE_MOTORCYCLES_BR_HISTEQ_PATH,
                                algorithm=equalize_histogram,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_ME_GRAYSCALE_PATH, 
                                save_path=SAVE_MOTORCYCLES_ME_HISTEQ_PATH,
                                algorithm=equalize_histogram,
                                algorithm_args=dict())

# Downscaling

In [41]:
SAVE_CARS_BR_DOWNSCALING_PATH = "./output/images/downscaling/cars-br" # Save path to downscaled images of cars with old license plate model
SAVE_CARS_ME_DOWNSCALING_PATH = "./output/images/downscaling/cars-me" # Save path to downscaled images of cars with MERCOSUL license plate model
SAVE_MOTORCYCLES_BR_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-br" # Save path to downscaled images of motorcycles with old license plate model
SAVE_MOTORCYCLES_ME_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-me" # Save path to downscaled images of motorcycles with MERCOSUL license plate model

DOWNSCALING_RATIO = 0.75

In [42]:
@verbose_decorator
def resize(image, ratio, interpolation_method_name, flag):
    width = int(image.shape[1] * ratio)
    height = int(image.shape[0] * ratio)
    dim = (width, height)

    if VERBOSE:
        print(f"interpolation method: {interpolation_method_name}")
        print(f"downscaling ratio: {str(ratio)}")
        print(f"original image shape: {image.shape}")
        print(f"new shape: {dim}")
        
    return cv2.resize(image, dim, interpolation=flag)

In [43]:
INTERPOLATION_FLAGS = [
    ('bilinear', cv2.INTER_LINEAR),
    ('bicubic', cv2.INTER_CUBIC),
    ('lanczos', cv2.INTER_LANCZOS4)
]

if not DEV_MODE and not os.path.exists("./output/images/downscaling/"): # If grayscale was not executed yet
    _ = sample_execute_save_show(data_path=SAVE_CARS_BR_HISTEQ_PATH, 
                                    save_path=SAVE_CARS_BR_DOWNSCALING_PATH,
                                    algorithm=resize, 
                                    algorithm_args={"ratio": DOWNSCALING_RATIO, "interpolation_method_name": INTERPOLATION_FLAGS[2][0], "flag": INTERPOLATION_FLAGS[2][1]})

    _ = sample_execute_save_show(data_path=SAVE_CARS_ME_HISTEQ_PATH, 
                                    save_path=SAVE_CARS_ME_DOWNSCALING_PATH,
                                    algorithm=resize, 
                                    algorithm_args={"ratio": DOWNSCALING_RATIO, "interpolation_method_name": INTERPOLATION_FLAGS[2][0], "flag": INTERPOLATION_FLAGS[2][1]})

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_BR_HISTEQ_PATH, 
                                    save_path=SAVE_MOTORCYCLES_BR_DOWNSCALING_PATH,
                                    algorithm=resize, 
                                    algorithm_args={"ratio": DOWNSCALING_RATIO, "interpolation_method_name": INTERPOLATION_FLAGS[2][0], "flag": INTERPOLATION_FLAGS[2][1]})

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_ME_HISTEQ_PATH, 
                                    save_path=SAVE_MOTORCYCLES_ME_DOWNSCALING_PATH,
                                    algorithm=resize, 
                                    algorithm_args={"ratio": DOWNSCALING_RATIO, "interpolation_method_name": INTERPOLATION_FLAGS[2][0], "flag": INTERPOLATION_FLAGS[2][1]})

# Noise Reduction

In [44]:
SAVE_CARS_BR_NOISEREDUCTION_PATH = "./output/images/noisereduction/cars-br" # Save path to noise reducted images of cars with old license plate model
SAVE_CARS_ME_NOISEREDUCTION_PATH = "./output/images/noisereduction/cars-me" # Save path to noise reducted images of cars with MERCOSUL license plate model
SAVE_MOTORCYCLES_BR_NOISEREDUCTION_PATH = "./output/images/noisereduction/motorcycles-br" # Save path to noise reducted images of motorcycles with old license plate model
SAVE_MOTORCYCLES_ME_NOISEREDUCTION_PATH = "./output/images/noisereduction/motorcycles-me" # Save path to noise reducted images of motorcycles with MERCOSUL license plate model

In [45]:
def reduce_noise(image, algorithm, algorithm_args):
    return algorithm(image, **algorithm_args)

In [46]:
if DEV_MODE:
    cv2.imwrite("./output/images/test/img_000082.jpg", cv2.imread("./dataset_RodoSol-ALPR/images/cars-br/img_000082.jpg"))

    # median blur
    unit_exec(image_path="./output/images/downscaling/cars-br/img_000082.jpg",
                save_path="./output/images/test/img_000082_medianblur.jpg",
                algorithm=cv2.medianBlur,
                algorithm_args={"ksize":5})
    
    # gaussian blur
    unit_exec(image_path="./output/images/downscaling/cars-br/img_000082.jpg",
            save_path="./output/images/test/img_000082_gaussianblur.jpg",
            algorithm=cv2.GaussianBlur,
            algorithm_args={"ksize": (5,5), "sigmaX": 0})
    
    # Non-Local Means (NLM) 
    unit_exec(image_path="./output/images/downscaling/cars-br/img_000082.jpg",
            save_path="./output/images/test/img_000082_NLM.jpg",
            algorithm=cv2.fastNlMeansDenoising,
            algorithm_args={"h": 5, "templateWindowSize": 7, "searchWindowSize": 21})
    
    


In [47]:
if not DEV_MODE and not os.path.exists("./output/images/noisereduction/"): # If grayscale was not executed yet
    _ = sample_execute_save_show(data_path=SAVE_CARS_BR_DOWNSCALING_PATH, 
                                    save_path=SAVE_CARS_BR_NOISEREDUCTION_PATH,
                                    algorithm=reduce_noise, 
                                    algorithm_args={"algorithm": cv2.fastNlMeansDenoising, "algorithm_args": {"h": 5, "templateWindowSize": 7, "searchWindowSize": 21}})

    _ = sample_execute_save_show(data_path=SAVE_CARS_ME_DOWNSCALING_PATH, 
                                    save_path=SAVE_CARS_ME_NOISEREDUCTION_PATH,
                                    algorithm=reduce_noise, 
                                    algorithm_args={"algorithm": cv2.fastNlMeansDenoising, "algorithm_args": {"h": 5, "templateWindowSize": 7, "searchWindowSize": 21}})

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_BR_DOWNSCALING_PATH, 
                                    save_path=SAVE_MOTORCYCLES_BR_NOISEREDUCTION_PATH,
                                    algorithm=reduce_noise, 
                                    algorithm_args={"algorithm": cv2.fastNlMeansDenoising, "algorithm_args": {"h": 5, "templateWindowSize": 7, "searchWindowSize": 21}})

    _ = sample_execute_save_show(data_path=SAVE_MOTORCYCLES_ME_DOWNSCALING_PATH, 
                                    save_path=SAVE_MOTORCYCLES_ME_NOISEREDUCTION_PATH,
                                    algorithm=reduce_noise, 
                                    algorithm_args={"algorithm": cv2.fastNlMeansDenoising, "algorithm_args": {"h": 5, "templateWindowSize": 7, "searchWindowSize": 21}})

# Comparing
Comparing original and processed images

In [48]:
SAVE_CARS_BR_COMPARING_PATH = "./output/images/comparing/cars-br"
SAVE_CARS_ME_COMPARING_PATH = "./output/images/comparing/cars-me"
SAVE_MOTORCYCLES_BR_COMPARING_PATH = "./output/images/comparing/motorcycles-br"
SAVE_MOTORCYCLES_ME_COMPARING_PATH = "./output/images/comparing/motorcycles-me"

In [49]:
original_images = sample_images(data_path=DATASET_CARS_BR_PATH, exclude_path="")
processed_images = sample_images(data_path=SAVE_CARS_BR_NOISEREDUCTION_PATH, exclude_path="")

compare_and_save(original_images, processed_images, SAVE_CARS_BR_COMPARING_PATH)


<function sample_images at 0x000001FADFF30DC0>
Excluded files from sampling: []
Sampled files: ['./dataset_RodoSol-ALPR/images/cars-br/img_000001.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000002.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000003.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000004.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000005.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000006.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000007.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000008.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000009.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000010.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000011.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000012.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000013.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000014.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000015.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000016.jpg', './data